# SEQUENCE-TO-SEQUENCE NMT WITH ATTENTION MECHANISM

# DATA

In [1]:
import numpy as np
import re
import string
import unicodedata
from unicodedata import normalize
import sys
import indicnlp
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
# from keras.models import Model
# from keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.utils import plot_model
import numpy as np
# !pip install tensorflow-addons==0.11.2
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import gensim
from tensorflow import keras
# !pip install indic-nlp-library : this is necessary, without this common wont be found and correct indic nlp wont get installed
import os
import io
import time
INDIC_NLP_LIB_HOME=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\anoopkunchukuttan-indic_nlp_library-eccde81"
INDIC_NLP_RESOURCES=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

# LOAD DATA

In [2]:
def load_doc(doc):
    file = open(doc , mode = 'rt' , encoding = 'utf -8')
    text = file.read()
    file.close()
    return text

def clean_english_data(lines):
    re_print = re.compile('[^%s]' %re.escape(string.printable))
    table = str.maketrans('' , '' , string.punctuation)
    
    
    clean_eng_lines = []
    ans = list()
    ans.append(lines)
    
    for line in ans:
        line = normalize('NFD' , line).encode('ascii' , 'ignore')
        line = line.decode('UTF-8')
        line = line.split()
        line = [word.lower() for word in line]
        line = [word.translate(table) for word in line]
        line = [word for word in line if word.isalpha()]
        line = ' '.join(line)
        line = '<start> ' + line +' <end>'
        clean_eng_lines.append(line)
    return clean_eng_lines

def clean_text(line):
    text = line
    text = text.replace(u',' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u'(' ,'')
    text = text.replace(u')' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u':' ,'')
    text = text.replace(u"'" ,'')
    text = text.replace(u"'" ,'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text = re.sub('[a-zA-Z]' , '' , text)
    text = re.sub('[0-9+\-*/.%&!]' , '' , text)
    return text

def clean_hindi_data(lines):
    clean_hin_lines = []
    ans = list()
    ans.append(lines)
    for line in ans:
        remove_nuktas = True
        factory = IndicNormalizerFactory()
        normalizer = factory.get_normalizer("hi" , remove_nuktas = True )
        line = clean_text(line)
        tokens = list()
        for t in indic_tokenize.trivial_tokenize(line):
            tokens.append(t)
        line = tokens
        line = [ word.lower() for word in line]
        line = [word for word in line if not re.search(r'\d', word)]
        line = ' '.join(line)
        line = '<start> ' + line +' <end>'
        clean_hin_lines.append(line)
        
    return clean_hin_lines   


def to_pairs(text):
    lines = text.strip().split('\n')
    
    eng_hin_pairs = [line.split('\t') for line in lines]
    eng_hin_pairs = np.delete(eng_hin_pairs, [0,1,2,3,4] , axis = 1)
    eng_hin_pairs = eng_hin_pairs.tolist()
    clean_data = list()
    for pair in eng_hin_pairs:
        temp_data = list()
        for i, sentence in enumerate(pair):
            if i == 0:
                x = clean_english_data(sentence)
                temp_data.append(x)
            if i == 1:
                y = clean_hindi_data(sentence)
                temp_data.append(y)
        clean_data.append(temp_data)
    return eng_hin_pairs, clean_data

doc = load_doc("hindi-visual-genome-train.txt")
pairs , clean_data = to_pairs(doc)

In [3]:
clean_data

[[['<start> it is an indoor scene <end>'],
  ['<start> यह एक इनडोर दृश्य है <end>']],
 [['<start> computer screens turned on <end>'],
  ['<start> कंप्यूटर स्क्रीन चालू <end>']],
 [['<start> man has short hair <end>'],
  ['<start> आदमी के छोटे बाल हैं <end>']],
 [['<start> photo album open on an adults lap <end>'],
  ['<start> एक वयस्क की गोद में फोटो एल्बम खुला <end>']],
 [['<start> there is a group of girls beside the black car <end>'],
  ['<start> काली कार के पास लड़कियों का एक समूह है <end>']],
 [['<start> child in a stroller <end>'],
  ['<start> एक घुमक्कड़ में बच्चा <end>']],
 [['<start> tall metal lightpost <end>'],
  ['<start> लंबा धातु प्रकाश पोस्ट <end>']],
 [['<start> wall is painted white <end>'],
  ['<start> दीवार सफेद रंग की है <end>']],
 [['<start> there are several pictures on the wall <end>'],
  ['<start> दीवार पर कई चित्र हैं <end>']],
 [['<start> woman facing the ocean <end>'],
  ['<start> सागर का सामना कर रही महिला <end>']],
 [['<start> this is an office layout <end>

In [4]:
# shuffling values
import random
BUFFER_SIZE = 28930
BATCH_SIZE = 64
num_examples = 20000
# train_dataset = clean_data[:n_train]
# val_dataset = clean_data[n_train:]
# # Let's limit the #training examples for faster training

# train_dataset = random.shuffle(train_dataset)
# val_dataset = random.shuffle(val_dataset)
# type(clean_data[0][1])

In [5]:
# train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [6]:
clean_hindi_lines = []
clean_english_lines = []
for i in range(len(clean_data)):
    clean_hindi_lines.append(clean_data[i][1])
    clean_english_lines.append(clean_data[i][0])
    


In [7]:
clean_hindi = list()
for i in range(len(clean_hindi_lines)):
    clean_hindi.append(clean_hindi_lines[i][0])
    
clean_english = list()
for i in range(len(clean_english_lines)):
    clean_english.append(clean_english_lines[i][0])


In [8]:
clean_english

['<start> it is an indoor scene <end>',
 '<start> computer screens turned on <end>',
 '<start> man has short hair <end>',
 '<start> photo album open on an adults lap <end>',
 '<start> there is a group of girls beside the black car <end>',
 '<start> child in a stroller <end>',
 '<start> tall metal lightpost <end>',
 '<start> wall is painted white <end>',
 '<start> there are several pictures on the wall <end>',
 '<start> woman facing the ocean <end>',
 '<start> this is an office layout <end>',
 '<start> four metallic chairs <end>',
 '<start> clutter is on a table <end>',
 '<start> a white microwave oven <end>',
 '<start> white suv driving through intersection <end>',
 '<start> person crossing street with umbrella <end>',
 '<start> man in gray pants leaning on building <end>',
 '<start> window on the building <end>',
 '<start> a man standing in between cars <end>',
 '<start> painting hanging on wall <end>',
 '<start> leg of the chair <end>',
 '<start> window of a boat <end>',
 '<start> th

In [9]:
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(padded_tensor_hin, padded_tensor_eng, test_size=0.2, random_state = 42)
input_train , input_val, target_train, target_val = train_test_split(clean_hindi , clean_english, test_size = 0.2, random_state = 42)

In [10]:
hin_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
hin_lang_tokenizer.fit_on_texts(clean_hindi)
hin_dict = hin_lang_tokenizer.word_index
tensor_hin = hin_lang_tokenizer.texts_to_sequences(input_train) 
padded_tensor_hin = tf.keras.preprocessing.sequence.pad_sequences(tensor_hin, padding='post')
# print(tensor)
# len(hin_dict)
input_tensor_train =  padded_tensor_hin
print(len(padded_tensor_hin))
print(len(hin_dict))

23144
7013


In [11]:
eng_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
eng_lang_tokenizer.fit_on_texts(clean_english)
eng_dict = eng_lang_tokenizer.word_index
tensor_eng = eng_lang_tokenizer.texts_to_sequences(target_train) 
padded_tensor_eng = tf.keras.preprocessing.sequence.pad_sequences(tensor_eng, padding='post')
target_tensor_train = padded_tensor_eng
# print(tensor)
# len(hin_dict)
print(len(padded_tensor_eng))
print(len(eng_dict))


23144
5553


In [12]:
# train test split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(padded_tensor_hin, padded_tensor_eng,  test_size=0.2, random_state = 42)


In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
# val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [14]:
train_dataset

<BatchDataset shapes: ((64, 34), (64, 26)), types: (tf.int32, tf.int32)>

In [15]:
len(target_tensor_train[0])

26

In [16]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 34]), TensorShape([64, 26]))

In [18]:
# some important parameters
vocab_inp_size = len(hin_lang_tokenizer.word_index)+1
vocab_tar_size = len(eng_lang_tokenizer.word_index)+1
max_length_input = len(input_tensor_train[0])
max_length_output = len(target_tensor_train[0])

embedding_dim = 100
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE

In [19]:
print("max_length_hindi, max_length_english, vocab_size_hindi, vocab_size_english")
max_length_input, max_length_output, vocab_inp_size, vocab_tar_size

max_length_hindi, max_length_english, vocab_size_hindi, vocab_size_english


(34, 26, 7014, 5554)

In [20]:
# hindi embedding word2vec
# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load("hi-d100-m2-cbow.model")

In [21]:
embeddings_dictionary_h = {}
for word in hin_lang_tokenizer.word_index:
    try:
        embeddings_dictionary_h[word] = model.wv.get_vector(word)
    except:
        pass
    
print("the no of key-value pairs:"  , len(embeddings_dictionary_h))
embeddings_dictionary_h

the no of key-value pairs: 6380


{'एक': array([ 1.3761369e+00,  3.1895986e-01,  5.5161637e-01,  3.3454773e+00,
         7.5480199e-01, -4.1311855e+00,  3.6096673e+00,  3.1984558e-03,
         1.1114923e+00,  1.8430928e+00, -1.6482291e+00, -2.6384809e+00,
        -2.3330653e+00, -1.3789964e+00,  2.8163950e+00,  2.2858524e+00,
         2.9680107e+00,  7.1689091e+00,  1.8727484e+00, -3.0235820e+00,
         2.2039354e+00, -1.5322008e+00,  4.8508924e-01,  1.1078984e+00,
        -3.8863792e+00, -1.7010270e-01, -2.9873648e-01, -1.1823559e+00,
        -3.1249235e+00,  2.6842210e-01,  2.3926444e-01, -2.8258526e+00,
        -7.1746130e+00, -8.4229892e-01, -2.8898644e+00,  3.0856481e-01,
        -2.0823562e+00,  5.0447965e-01, -7.3356193e-01,  2.7815120e+00,
         7.8827173e-01,  8.9215755e-01, -1.7951288e+00,  1.1564068e+00,
         5.7838392e-02,  3.4770534e+00, -1.4091858e+00, -2.1184385e+00,
        -3.9627492e+00, -3.0346465e-01, -1.0447567e+00, -1.7222787e+00,
        -1.6216998e+00, -2.7504045e-01,  2.9573914e-02, -2

In [22]:
num_words = vocab_inp_size
embedding_matrix_h = zeros((vocab_inp_size, 100))
for word, index in hin_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary_h.get(word)
    if embedding_vector is not None:
        embedding_matrix_h[index] = embedding_vector

In [23]:
len(embedding_matrix_h)

7014

In [24]:
# preparing english lines for w2v
eng = []
for line in clean_english_lines:
    for l in line:
        l = l.split(" ")
    eng.append(l)

eng

[['<start>', 'it', 'is', 'an', 'indoor', 'scene', '<end>'],
 ['<start>', 'computer', 'screens', 'turned', 'on', '<end>'],
 ['<start>', 'man', 'has', 'short', 'hair', '<end>'],
 ['<start>', 'photo', 'album', 'open', 'on', 'an', 'adults', 'lap', '<end>'],
 ['<start>',
  'there',
  'is',
  'a',
  'group',
  'of',
  'girls',
  'beside',
  'the',
  'black',
  'car',
  '<end>'],
 ['<start>', 'child', 'in', 'a', 'stroller', '<end>'],
 ['<start>', 'tall', 'metal', 'lightpost', '<end>'],
 ['<start>', 'wall', 'is', 'painted', 'white', '<end>'],
 ['<start>',
  'there',
  'are',
  'several',
  'pictures',
  'on',
  'the',
  'wall',
  '<end>'],
 ['<start>', 'woman', 'facing', 'the', 'ocean', '<end>'],
 ['<start>', 'this', 'is', 'an', 'office', 'layout', '<end>'],
 ['<start>', 'four', 'metallic', 'chairs', '<end>'],
 ['<start>', 'clutter', 'is', 'on', 'a', 'table', '<end>'],
 ['<start>', 'a', 'white', 'microwave', 'oven', '<end>'],
 ['<start>', 'white', 'suv', 'driving', 'through', 'intersection', '

In [25]:
### english word embeddings
model_e = gensim.models.Word2Vec(eng , vector_size = 100 , min_count = 1 , window = 2)

In [26]:
model_e.train(eng , epochs = 50 , total_examples = len(clean_english_lines))

(5120776, 10040650)

In [27]:
model_e.wv.get_vector('<start>')

array([ 0.87490296, -0.09882154, -0.05467283,  0.03689966, -1.0424775 ,
        0.2451949 ,  0.4816406 ,  1.3944693 , -0.9768243 , -0.591232  ,
        0.6522363 , -0.31312212,  0.46150634, -0.11188674, -0.02159368,
       -1.1506121 ,  0.7444597 ,  1.0072037 , -0.49099204, -1.1490719 ,
       -0.50322735, -1.1848322 ,  0.03314469,  0.2900186 ,  1.3064814 ,
       -0.8469717 , -0.13971807,  0.12079688,  0.5186532 , -0.9081379 ,
        1.1348873 ,  0.85046375, -0.03409245, -0.68661934, -0.7269997 ,
       -0.54655755,  0.5903339 , -0.6464924 , -0.79427814, -0.33710992,
        1.146509  , -1.4263941 ,  0.18529236, -0.56153905, -0.20012774,
       -0.5727961 , -0.7474901 , -0.22580644, -1.2874714 ,  0.14173217,
        0.83859473,  0.94332904,  1.6359757 , -0.14351793, -0.7600844 ,
       -0.3296438 ,  0.03717903,  0.8460593 ,  0.2771263 , -0.4611981 ,
       -0.47301716, -0.27608725,  0.5962553 ,  0.00978156,  0.05369976,
        0.17065229, -0.02104025,  0.03365793, -0.08932974, -0.65

In [28]:
embeddings_dictionary = {}
for word in eng_lang_tokenizer.word_index:
    try:
        embeddings_dictionary[word] = model_e.wv.get_vector(word)
    except:
        pass
    
print("the no of key-value pairs:"  , len(embeddings_dictionary))
embeddings_dictionary

the no of key-value pairs: 5552


{'<start>': array([ 0.87490296, -0.09882154, -0.05467283,  0.03689966, -1.0424775 ,
         0.2451949 ,  0.4816406 ,  1.3944693 , -0.9768243 , -0.591232  ,
         0.6522363 , -0.31312212,  0.46150634, -0.11188674, -0.02159368,
        -1.1506121 ,  0.7444597 ,  1.0072037 , -0.49099204, -1.1490719 ,
        -0.50322735, -1.1848322 ,  0.03314469,  0.2900186 ,  1.3064814 ,
        -0.8469717 , -0.13971807,  0.12079688,  0.5186532 , -0.9081379 ,
         1.1348873 ,  0.85046375, -0.03409245, -0.68661934, -0.7269997 ,
        -0.54655755,  0.5903339 , -0.6464924 , -0.79427814, -0.33710992,
         1.146509  , -1.4263941 ,  0.18529236, -0.56153905, -0.20012774,
        -0.5727961 , -0.7474901 , -0.22580644, -1.2874714 ,  0.14173217,
         0.83859473,  0.94332904,  1.6359757 , -0.14351793, -0.7600844 ,
        -0.3296438 ,  0.03717903,  0.8460593 ,  0.2771263 , -0.4611981 ,
        -0.47301716, -0.27608725,  0.5962553 ,  0.00978156,  0.05369976,
         0.17065229, -0.02104025,  0.033

In [29]:
num_words = vocab_tar_size
embedding_matrix = zeros((vocab_tar_size, 100))
for word, index in eng_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [30]:
class Encoder(tf.keras.Model):
    
    
    def __init__(self,embedding_matrix_h, vocab_size, embedding_dim, enc_units, batch_sz):
        
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        embedding_matrix_h = embedding_matrix_h
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [embedding_matrix_h])

        ##-------- LSTM layer in Encoder ------- ##
        self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')



    def call(self, x, hidden):
        
        x = self.embedding(x)
        output, h, c = self.lstm_layer(x, initial_state = hidden)
        return output, h, c

    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [31]:
## Test Encoder Stack

encoder = Encoder(embedding_matrix_h, vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (64, 34, 1024)
Encoder h vecotr shape: (batch size, units) (64, 1024)
Encoder c vector shape: (batch size, units) (64, 1024)


In [32]:
class Decoder(tf.keras.Model):
    def __init__(self, embedding_matrix, vocab_size, embedding_dim, dec_units, batch_sz, attention_type = 'luong'):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.attention_type = attention_type
        
        embedding_matrix = embedding_matrix
        
        #Embedding layer
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [embedding_matrix])
        
        #final dense layer on which softmax will be applied
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # define the fundamental cell for decoder recurrent structure
        
        self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
        
        # sampler 
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        
        # create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(self.dec_units, None, self.batch_sz*[max_length_input], self.attention_type)
        
        # wrap attention mechanism with the fundamental rnn cell
        self.rnn_cell = self.build_rnn_cell(batch_sz)
        
        # define the decoder with respect to fundamental rnn cell
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

        
        
    def build_rnn_cell(self, batch_sz):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                      self.attention_mechanism, attention_layer_size=self.dec_units)
        return rnn_cell
    
    def build_attention_mechanism(self, dec_units, memory,memory_sequence_length, attention_type = 'luong'):
         # ------------- #
        # typ: Which sort of attention (Bahdanau, Luong)
        # dec_units: final dimension of attention outputs 
        # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
        # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)
        if(attention_type == 'bahdanau'):
            return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
        
        else:
             return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
            
    def build_initial_state(self, batch_sz, encoder_state, Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
        return decoder_initial_state


    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
        return outputs        

        
    

In [33]:
# Test decoder stack

decoder = Decoder(embedding_matrix, vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)

Decoder Outputs Shape:  (64, 25, 5554)


In [34]:
# defing optimizer and loss function

optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
    # real shape = (BATCH_SIZE, max_length_output)
    # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
    cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = cross_entropy(y_true=real, y_pred=pred)
    mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)  
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [35]:

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [36]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


        dec_input = targ[ : , :-1 ] # Ignore <end> token
        real = targ[ : , 1: ]         # ignore <start> token

        # Set the AttentionMechanism object with encoder_outputs
        decoder.attention_mechanism.setup_memory(enc_output)

        # Create AttentionWrapperState as initial_state for decoder
        decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
        pred = decoder(dec_input, decoder_initial_state)
        logits = pred.rnn_output
        loss = loss_function(real, logits)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [37]:
# training the model
# EPOCHS = 10

# for epoch in range(EPOCHS):
#     start = time.time()
#     enc_hidden = encoder.initialize_hidden_state()
#     total_loss = 0
#     # print(enc_hidden[0].shape, enc_hidden[1].shape)
    
#     for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
#         batch_loss = train_step(inp, targ, enc_hidden)
#         total_loss += batch_loss
#         if batch % 100 == 0:
#             print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
#                                                    batch,
#                                                    batch_loss.numpy()))
#    # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

#     print('Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                       total_loss / steps_per_epoch))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.1020
Epoch 1 Batch 100 Loss 0.7934
Epoch 1 Batch 200 Loss 0.6439
Epoch 1 Batch 300 Loss 0.4603
Epoch 1 Loss 0.7323
Time taken for 1 epoch 649.7831158638 sec

Epoch 2 Batch 0 Loss 0.5146
Epoch 2 Batch 100 Loss 0.4413
Epoch 2 Batch 200 Loss 0.5197
Epoch 2 Batch 300 Loss 0.3546
Epoch 2 Loss 0.4058
Time taken for 1 epoch 637.4984674453735 sec

Epoch 3 Batch 0 Loss 0.3680
Epoch 3 Batch 100 Loss 0.2962
Epoch 3 Batch 200 Loss 0.3073
Epoch 3 Batch 300 Loss 0.2678
Epoch 3 Loss 0.2975
Time taken for 1 epoch 638.290522813797 sec

Epoch 4 Batch 0 Loss 0.2162
Epoch 4 Batch 100 Loss 0.2503
Epoch 4 Batch 200 Loss 0.1598
Epoch 4 Batch 300 Loss 0.2786
Epoch 4 Loss 0.2271
Time taken for 1 epoch 642.9178221225739 sec

Epoch 5 Batch 0 Loss 0.1854
Epoch 5 Batch 100 Loss 0.1587
Epoch 5 Batch 200 Loss 0.2630
Epoch 5 Batch 300 Loss 0.2810
Epoch 5 Loss 0.2148
Time taken for 1 epoch 642.9728994369507 sec

Epoch 6 Batch 0 Loss 0.2096
Epoch 6 Batch 100 Loss 0.2119
Epoch 6 Batch 200 Loss 0.1

In [37]:
def evaluate_sentence(sentence):
    sentence = clean_hindi_data(sentence)
    string = " ".join(sentence)

    inputs = [hin_lang_tokenizer.word_index[i] for i in string.strip().split(" ")]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''
    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)
    dec_h = enc_h
    dec_c = enc_c
    start_tokens = tf.fill([inference_batch_size], eng_lang_tokenizer.word_index['<start>'])
    end_token = eng_lang_tokenizer.word_index['<end>']
    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
    # Instantiate BasicDecoder object
    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
    # Setup Memory in decoder stack
    decoder.attention_mechanism.setup_memory(enc_out)

    # set decoder_initial_state
    decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


    ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
    ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
    ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function
    decoder_embedding_matrix = decoder.embedding.variables[0]
    outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
    return outputs.sample_id.numpy()

def translate(sentence):
    result = evaluate_sentence(sentence)
    print(result)
    result = eng_lang_tokenizer.sequences_to_texts(result)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    return result, sentence

In [39]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [40]:
input_val

['<start> यह आकाश है <end>',
 '<start> बैंगनी और सफेद जूते के साथ लड़का <end>',
 '<start> डैपर लाल टोपी पहने हुए आदमी <end>',
 '<start> हरे रंग के पत्ते <end>',
 '<start> एक धावक पहले बेस से आगे निकल गया <end>',
 '<start> एक लाल रंग की ट्रेन <end>',
 '<start> अंधेरे लाल इमारत पर विचलित <end>',
 '<start> आदमी पर काला सूट जैकेट <end>',
 '<start> बड़ा धूसर भारतीय हाथी <end>',
 '<start> आदमी के पास एक अंगूठी है <end>',
 '<start> लड़की ने हार पहना हुआ है <end>',
 '<start> बोरहोल से पानी खींचता आदमी <end>',
 '<start> सड़क संकेत <end>',
 '<start> इमारत पर खिडकि <end>',
 '<start> मैदान में बैंगनी फूलों के शीर्ष <end>',
 '<start> फोन पकड़े हुए एक महीला <end>',
 '<start> सफेद प्लग के साथ काले आइपॉड <end>',
 '<start> ज़ेबरा पर धारियाँ <end>',
 '<start> काले जैतून का एक कटा हुआ टुकड़ा <end>',
 '<start> गाय लेटी हुई थी <end>',
 '<start> लकड़ी के स्लेट के साथ लकड़ी की बेंच <end>',
 '<start> यह एक इमारत है <end>',
 '<start> महिलाएं कुर्सी पर बैठी हैं <end>',
 '<start> प्लेट पर कटा हुआ लाल टमाटर <end>'

In [41]:
target_val

['<start> this is the sky <end>',
 '<start> boy with purple and white shoes <end>',
 '<start> man wearing dapper red hat <end>',
 '<start> green colored leaves <end>',
 '<start> a runner leading off of first base <end>',
 '<start> a red train <end>',
 '<start> windown on dark red building <end>',
 '<start> black suit jacket on man <end>',
 '<start> big grey indian elephant <end>',
 '<start> man has a ring on <end>',
 '<start> the girl is wearing a necklace <end>',
 '<start> man drawing water from borehole <end>',
 '<start> street sign <end>',
 '<start> a window on the building <end>',
 '<start> tops of purple flowers in the field <end>',
 '<start> woman holding a cell phone <end>',
 '<start> black ipod with white plug <end>',
 '<start> stripes on the zebra <end>',
 '<start> a sliced piece of black olive <end>',
 '<start> the cow lying down <end>',
 '<start> wooden bench with wooden slats <end>',
 '<start> this is a buiding <end>',
 '<start> women are sitting on the chair <end>',
 '<sta

In [42]:
# removing <start> and <end> from input_val
input_val_rem = []
for i in range(len(input_val)):
    l = input_val[i].split(" ")
    l.pop(0)
    n = len(l)-1
    l.pop(n)
    s = " ".join(l)
    input_val_rem.append(s)
    

In [43]:
input_val_rem.index('')

1346

In [44]:
input_val_rem.pop(1346)
target_val.pop(1346)

'<start> tabby cat resting under desk <end>'

In [45]:
input_val_rem.pop(1655)
target_val.pop(1655)

'<start> tall giraffe standing in dirt <end>'

In [46]:
input_val_rem.pop(4508)
target_val.pop(4508)

'<start> the ramp says supreme <end>'

In [47]:
print(len(input_val_rem) , len(target_val))

5783 5783


In [48]:
input_val_rem.pop(258)
target_val.pop(258)
print(len(input_val_rem) , len(target_val))

5782 5782


In [50]:
# input_val_rem.index('')

In [51]:
translation = []
for i in range(0,100):
    translated_sent = translate(input_val_rem[i])
    translation.append(translated_sent)

[[17  8  5 27  3]]
Input: यह आकाश है
Predicted translation: ['this is the sky <end>']
[[ 61  14   4  20  12  10 269   3]]
Input: बैंगनी और सफेद जूते के साथ लड़का
Predicted translation: ['boy with a brown and white shoes <end>']
[[11 23  4 15 93  3]]
Input: डैपर लाल टोपी पहने हुए आदमी
Predicted translation: ['man wearing a red hat <end>']
[[ 19 213   3]]
Input: हरे रंग के पत्ते
Predicted translation: ['green leaves <end>']
[[  4 230  14 196 189   3]]
Input: एक धावक पहले बेस से आगे निकल गया
Predicted translation: ['a skier with right out <end>']
[[ 4 15 26  3]]
Input: एक लाल रंग की ट्रेन
Predicted translation: ['a red train <end>']
[[15 31  6 31  3]]
Input: अंधेरे लाल इमारत पर विचलित
Predicted translation: ['red building on building <end>']
[[ 13 286   6 140  11   3]]
Input: आदमी पर काला सूट जैकेट
Predicted translation: ['black suit on mans man <end>']
[[  37   99   75    9 4008    3]]
Input: बड़ा धूसर भारतीय हाथी
Predicted translation: ['large gray elephant in strides <end>']
[[  5  11 

[[ 11 345   4 120   3]]
Input: सर्फबोर्ड ले जाने वाला आदमी
Predicted translation: ['man carrying a surfboard <end>']
[[ 68   6 268  63   3]]
Input: पहाड़ की चोटी पर बर्फ
Predicted translation: ['snow on mountain top <end>']
[[  5  83   9   5 939   3]]
Input: शॉट में पक्षी
Predicted translation: ['the bird in the shot <end>']
[[17  8  4 11  3]]
Input: यह एक आदमी है
Predicted translation: ['this is a man <end>']
[[75 35  9  5 75  3]]
Input: हाथी समाशोधन में खड़ा है
Predicted translation: ['elephant standing in the elephant <end>']
[[42  6  5 42  3]]
Input: घड़ी पर यूडी
Predicted translation: ['clock on the clock <end>']
[[ 18 133   4 306   7  78   3]]
Input: पिज्जा का एक टुकड़ा खा रही महिला
Predicted translation: ['woman eating a slice of pizza <end>']
[[  4 272   8  35   9   4 272   3]]
Input: सर्विसमैन एक लाइन में खड़ा है
Predicted translation: ['a line is standing in a line <end>']
[[39  7  4 21  3]]
Input: एक व्यक्ति का सिर
Predicted translation: ['head of a person <end>']
[[ 4 60  8

In [52]:
target_val_1 = target_val[:100]
for i in range(len(target_val_1)):
    target_val_1[i] = target_val_1[i].split(" ")[1:-1]
    
t = []
for i in range(len(translation)):
    t.append(translation[i][0]) 
    
t1=[]
for i in range(len(t)):
    l = t[i][0].split(" ")[:-1]
    s = " ".join(l)
    t1.append(s)

In [54]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
avg_1g_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
#     print(score)
    avg_1g_score = avg_1g_score + score

avg_1g_score = avg_1g_score/len(t1)
avg_1g_score

0.7247592942765708

In [55]:
avg_2g_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(0, 1, 0, 0))
#     print(score)
    avg_2g_score = avg_2g_score + score

avg_2g_score = avg_2g_score/len(t1)
avg_2g_score

0.5515799625282738

In [56]:
avg_ng_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
#     print(score)
    avg_ng_score = avg_ng_score + score

avg_ng_score = avg_ng_score/len(t1)
avg_ng_score

0.33348933905951983